# **Generación del Conjunto de Datos de Origen**

**Objetivo:** El primer paso en cualquier proyecto de datos es tener datos. En esta celda, simulamos el proceso de recibir datos crudos de los distintos sistemas de una empresa de e-commerce. En lugar de usar un archivo CSV estático, generamos los datos mediante programación para crear un escenario dinámico y realista.

---
## **Simulando un Ecosistema de Datos Real**

En el mundo real, los datos no provienen de un solo lugar. Llegan de múltiples fuentes:

* Un sistema **CRM** para la gestión de usuarios.
* Un **ERP** para el manejo de proveedores y productos.
* Una **plataforma de ventas** que registra los pedidos.
* Un **sistema de logística** para gestionar las devoluciones.

Este script utiliza Python, PySpark y la librería **`Faker`** para imitar este ecosistema, creando cinco DataFrames distintos que representan estas fuentes.



---
## **Componentes Clave del Script**

* **`Faker`**: Es una potente librería de Python que nos permite crear datos falsos pero realistas (nombres, empresas, ciudades, etc.). Es una herramienta fundamental para probar pipelines de datos sin exponer información sensible.

* **La Semilla de la Reproducibilidad (`SEED = 42`) 🌱**: Este es uno de los conceptos más importantes de la celda. Al fijar una "semilla", nos aseguramos de que el generador de números "aleatorios" produzca **siempre la misma secuencia**. Esto significa que, sin importar cuántas veces ejecutemos el código, los datos generados serán idénticos. Para un taller, esto es crucial, ya que garantiza que todos los participantes trabajen con el mismo conjunto de datos y obtengan los mismos resultados.

* **Las Entidades de Negocio Creadas 📈**: Hemos modelado cinco entidades clave para nuestro negocio simulado, lo que nos dará material para análisis interesantes:
    * **`Proveedores`**: ¿De dónde vienen nuestros productos?
    * **`Usuarios`**: ¿Quiénes son nuestros clientes?
    * **`Productos`**: ¿Qué vendemos? (Ahora vinculados a un proveedor).
    * **`Pedidos`**: El registro de las ventas, el corazón de nuestro negocio.
    * **`Devoluciones`**: ¿Qué productos se devuelven y por qué?

Al finalizar la ejecución de esta celda, tendremos cinco DataFrames de PySpark cargados en la memoria del clúster, listos para ser procesados e ingeridos en nuestra arquitectura Medallion.

In [0]:
# Celda 1: Script para generar un conjunto de datos de e-commerce enriquecido

from pyspark.sql import SparkSession
from faker import Faker
import random
from datetime import datetime, timedelta

# --- SEMILLA PARA REPRODUCIBILIDAD ---
# Garantiza que los datos generados sean siempre los mismos.
SEED = 42
Faker.seed(SEED)
random.seed(SEED)

# Inicializar Faker y Spark Session
fake = Faker('es_ES')
# spark = SparkSession.builder.appName("Taller de Arquitecturas de Datos").getOrCreate() # En Databricks, 'spark' ya está disponible

# --- 1. GENERACIÓN DE PROVEEDORES ---
def generar_proveedores(n=10):
    """Genera una lista de proveedores de productos."""
    return [{'id_proveedor': 500 + i, 'nombre_proveedor': fake.company()} for i in range(n)]

# --- 2. GENERACIÓN DE USUARIOS ---
def generar_usuarios(n=100):
    """Genera una lista de diccionarios de usuarios."""
    return [{'id_usuario': 1000 + i, 'nombre': fake.name(), 'email': fake.email(), 'fecha_registro': fake.date_time_between(start_date='-2y'), 'ciudad': fake.city()} for i in range(n)]

# --- 3. GENERACIÓN DE PRODUCTOS (Ahora con proveedor) ---
def generar_productos(proveedores, n=50):
    """Genera una lista de productos, asignando un proveedor a cada uno."""
    categorias = ['Electrónica', 'Hogar', 'Ropa', 'Libros', 'Deportes']
    data = []
    for i in range(n):
        data.append({
            'id_producto': 2000 + i,
            'id_proveedor': random.choice(proveedores)['id_proveedor'],
            'nombre_producto': fake.word().capitalize() + " " + fake.word(),
            'categoria': random.choice(categorias),
            'precio_unitario': round(random.uniform(5.0, 250.0), 2)
        })
    return data

# --- 4. GENERACIÓN DE PEDIDOS (Ahora con método de pago) ---
def generar_pedidos(usuarios, productos, n=500):
    """Genera una lista de pedidos, vinculando usuarios y productos."""
    metodos_pago = ['Tarjeta de Crédito', 'PayPal', 'PSE', 'Efectivo']
    data = []
    for i in range(n):
        usuario = random.choice(usuarios)
        producto = random.choice(productos)
        cantidad = random.randint(1, 5)
        data.append({
            'id_pedido': 3000 + i,
            'id_usuario': usuario['id_usuario'],
            'id_producto': producto['id_producto'],
            'cantidad': cantidad,
            'monto': round(cantidad * producto['precio_unitario'], 2),
            'metodo_pago': random.choice(metodos_pago),
            'fecha_pedido': fake.date_time_between(start_date=usuario['fecha_registro'])
        })
    return data

# --- 5. GENERACIÓN DE DEVOLUCIONES ---
def generar_devoluciones(pedidos, n=30):
    """Genera una lista de devoluciones, seleccionando pedidos al azar."""
    motivos = ['Producto defectuoso', 'No era la talla correcta', 'No cumple expectativas', 'Me arrepentí de la compra']
    pedidos_devueltos = random.sample(pedidos, n)
    data = []
    for i, pedido in enumerate(pedidos_devueltos):
        data.append({
            'id_devolucion': 7000 + i,
            'id_pedido': pedido['id_pedido'],
            'fecha_devolucion': fake.date_time_between(start_date=pedido['fecha_pedido']),
            'motivo': random.choice(motivos)
        })
    return data

# --- CREACIÓN DE DATAFRAMES EN MEMORIA ---
print("Generando datos simulados...")
proveedores_data = generar_proveedores(10)
usuarios_data = generar_usuarios(100)
productos_data = generar_productos(proveedores_data, 50)
pedidos_data = generar_pedidos(usuarios_data, productos_data, 500)
devoluciones_data = generar_devoluciones(pedidos_data, 30)

proveedores_df = spark.createDataFrame(proveedores_data)
usuarios_df = spark.createDataFrame(usuarios_data)
productos_df = spark.createDataFrame(productos_data)
pedidos_df = spark.createDataFrame(pedidos_data)
devoluciones_df = spark.createDataFrame(devoluciones_data)

print("\n--- DataFrames Creados en Memoria ---")
print(f"Proveedores: {proveedores_df.count()} registros")
print(f"Usuarios: {usuarios_df.count()} registros")
print(f"Productos: {productos_df.count()} registros")
print(f"Pedidos: {pedidos_df.count()} registros")
print(f"Devoluciones: {devoluciones_df.count()} registros")

print("\nCelda 1 completada: El conjunto de datos enriquecido está listo para ser ingerido en la capa Bronce.")

# Ingesta a la Capa BRONCE - Persistencia de los Datos de Origen

**Objetivo:** El propósito de esta celda es tomar los datos que existen temporalmente en la memoria del clúster (los DataFrames de la Celda 1) y guardarlos como tablas permanentes. Este es el primer paso oficial de nuestro pipeline: la materialización de los datos en la capa **Bronce**.

---
## **El Archivo de Datos Crudos**

La capa Bronce funciona como el **archivo histórico y fiel** de nuestros datos de origen. Su principal y única regla es almacenar la información **exactamente como fue recibida**, sin aplicar ninguna limpieza, transformación o validación.

Este enfoque nos proporciona dos ventajas estratégicas:

1.  **Auditabilidad y Linaje**: Mantenemos un registro perfecto del estado de los datos en el momento de su ingesta. Esto es crucial para auditorías y para rastrear el origen de cualquier dato en las capas posteriores.
2.  **Capacidad de Reprocesamiento**: Si en el futuro las reglas de negocio cambian, siempre podemos volver a esta capa Bronce intacta y volver a ejecutar nuestros pipelines de transformación desde el principio con la nueva lógica, sin tener que volver a conectarnos a los sistemas de origen.



---
## **Componentes Clave del Script**

* **Formato Delta Lake**: No guardamos los datos en un formato simple como CSV. Usamos **Delta Lake**, el estándar en Databricks. Esto convierte nuestro almacenamiento en un sistema transaccional robusto, dándonos garantías **ACID** (los trabajos no quedan a medias), un rendimiento optimizado y la capacidad de "viajar en el tiempo" a versiones anteriores de los datos.

* **`saveAsTable()`**: Este comando es el corazón de la celda. Realiza dos acciones clave:
    1.  **Guarda los datos** en el almacenamiento subyacente (como archivos Parquet optimizados).
    2.  **Registra la tabla** en el catálogo de Databricks (Unity Catalog o Hive Metastore), lo que la hace visible y consultable mediante SQL.

* **Organización (`USE curso_arquitecturas`)**: Antes de guardar, nos aseguramos de estar en la base de datos correcta. Mantener las tablas de un proyecto dentro de su propio `schema` o base de datos es una práctica esencial para tener un espacio de trabajo ordenado.

Al finalizar la ejecución de esta celda, nuestros datos ya no son temporales. Se han convertido en cinco tablas Delta persistentes que forman la base de nuestra arquitectura, listas para ser refinadas en la siguiente capa: Plata.

In [0]:
# Celda 2: Guardar los DataFrames como Tablas Delta en la Capa Bronce

# --- 1. Definir y usar la base de datos para nuestro proyecto ---
db_name = "curso_arquitecturas"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
spark.sql(f"USE {db_name}")

print(f"Usando la base de datos: {db_name}\n")

# --- 2. Persistir cada DataFrame como una tabla Delta en la capa Bronce ---

# Guardar la tabla de proveedores
proveedores_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("proveedores_bronze")
print("Tabla 'proveedores_bronze' guardada exitosamente.")

# Guardar la tabla de usuarios
usuarios_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("usuarios_bronze")
print("Tabla 'usuarios_bronze' guardada exitosamente.")

# Guardar la tabla de productos
productos_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true")\
    .saveAsTable("productos_bronze")
print("Tabla 'productos_bronze' guardada exitosamente.")

# Guardar la tabla de pedidos
pedidos_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true")\
    .saveAsTable("pedidos_bronze")
print("Tabla 'pedidos_bronze' guardada exitosamente.")

# Guardar la tabla de devoluciones
devoluciones_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("devoluciones_bronze")
print("Tabla 'devoluciones_bronze' guardada exitosamente.")

print("\n¡Proceso completado! Las 5 tablas de la capa Bronce ya están disponibles en el catálogo para la siguiente etapa.")

# **Celda 2.5 (Explicativa): Propiedades ACID y Auditoría de Datos en Delta Lake**

Antes de proceder con la transformación de datos a la capa Plata, es importante comprender una característica fundamental de las tablas Delta que acabamos de crear: las garantías transaccionales **ACID**.

-----

## **Garantías Transaccionales ACID**

ACID es un acrónimo que designa un conjunto de cuatro propiedades que aseguran la fiabilidad e integridad de los datos durante las operaciones de escritura.

  * **Atomicidad (A)**: Asegura que cada transacción se trate como una única unidad de trabajo que se ejecuta completamente o no se ejecuta en absoluto. Su principal beneficio es la prevención de la corrupción de datos por escrituras parciales.

  * **Consistencia (C)**: Garantiza que cada transacción lleva los datos de un estado válido a otro. Se preserva la integridad de la base de datos según las reglas definidas.

  * **Aislamiento (I)**: Asegura que las transacciones concurrentes se ejecuten de manera independiente, sin interferir entre sí. Esto previene inconsistencias al leer datos que están siendo modificados simultáneamente.

  * **Durabilidad (D)**: Una vez que una transacción se ha completado, sus cambios son permanentes y persistirán incluso en caso de fallos del sistema.

En conjunto, estas propiedades otorgan al Data Lake la fiabilidad de los sistemas de bases de datos tradicionales, lo cual es indispensable en entornos de producción.

-----

## **Auditoría y Versionado de Datos**

Estas garantías son posibles gracias al **registro de transacciones** (transaction log) que Delta Lake mantiene para cada tabla. Este registro es consultable y proporciona una completa auditoría de todas las operaciones realizadas.

El comando `DESCRIBE HISTORY` permite inspeccionar este historial.

**Código:**

```sql
-- Consultar el historial de transacciones de la tabla de pedidos.
DESCRIBE HISTORY curso_arquitecturas.pedidos_bronze;
```

**Análisis del Resultado:**

La salida de este comando detalla cada operación (versión, marca de tiempo, tipo de operación, parámetros y usuario) que ha modificado la tabla.

Esta capacidad de versionado es la base para la funcionalidad de "Time Travel", que permite consultar estados anteriores de los datos para fines de depuración, auditoría o restauración.

In [0]:
%sql
DESCRIBE HISTORY curso_arquitecturas.pedidos_bronze;

### **Ejemplo 1: Viaje en el Tiempo por Número de Versión (`VERSION AS OF`)**

La forma más directa de viajar en el tiempo es pidiendo una versión específica. Como nuestra tabla `pedidos_bronze` se creó en una sola operación (`WRITE`), solo tiene la versión 0 y la 1 (la creación y la escritura). Vamos a simular un cambio para poder comparar.

In [0]:
%sql
-- Primero, vamos a simular un cambio: borremos los pedidos del método de pago 'Efectivo'.
-- Esto creará la versión 2 de la tabla.
DELETE FROM curso_arquitecturas.pedidos_bronze WHERE metodo_pago = 'Efectivo';

-- Ahora, comparemos la versión más reciente (2) con la anterior (1).
SELECT 
  'Version 1 (Antes del borrado)' AS version, 
  COUNT(*) AS total_filas 
FROM curso_arquitecturas.pedidos_bronze VERSION AS OF 1
UNION ALL
SELECT 
  'Version 2 (Después del borrado)' AS version, 
  COUNT(*) AS total_filas 
FROM curso_arquitecturas.pedidos_bronze;

**Caso de uso:** Perfecto para restaurar una tabla a un estado anterior después de un error, como un borrado accidental.

### **Ejemplo 2: Viaje en el Tiempo por Marca de Tiempo (`TIMESTAMP AS OF`)**

A veces no conoces el número de la versión, pero sí sabes la hora en que los datos estaban correctos. Puedes usar una marca de tiempo para consultar. La cadena de fecha y hora debe estar en el formato estándar.

In [0]:
%sql
-- Consulta el estado de la tabla de usuarios ANTES de que ejecutáramos este notebook hoy.
-- Reemplaza con una marca de tiempo anterior a la ejecución de la celda 2.
SELECT 
  fecha_registro,
  nombre
FROM curso_arquitecturas.usuarios_bronze TIMESTAMP AS OF "2025-08-13T23:00:00.000+0000" -- Ajusta la fecha y hora a tu zona si es necesario
LIMIT 5;


**Caso de uso:** Investigar el estado de los datos justo antes de un despliegue o una ejecución de pipeline que pudo haber causado un problema.

### **Ejemplo 3: Ver los Cambios Entre Dos Versiones (`table_changes`)**

Esta es una función de tabla increíblemente útil para la auditoría. Te muestra exactamente qué filas cambiaron entre dos versiones.


In [0]:
%sql
-- **Paso 1: Habilitar Change Data Feed con la sintaxis correcta.**
-- El nombre de la propiedad es 'delta.enableChangeDataFeed' (sin guion).
ALTER TABLE curso_arquitecturas.pedidos_bronze SET TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true');

In [0]:
%sql
-- **Paso 2: Realizar una nueva operación para generar un cambio que sea rastreado.**
-- CDF solo rastrea los cambios OCURRIDOS DESPUÉS de su habilitación.
-- Vamos a borrar los pedidos pagados con 'PayPal' para generar un nuevo historial.
DELETE FROM curso_arquitecturas.pedidos_bronze WHERE metodo_pago = 'PayPal';

In [0]:
%sql
-- **Paso 3: (Opcional pero recomendado) Consultar el historial para obtener los números de versión.**
DESCRIBE HISTORY curso_arquitecturas.pedidos_bronze;
-- Anota la versión de la operación 'DELETE' y la versión inmediatamente anterior.

In [0]:
%sql
-- **Paso 4: Ejecutar 'table_changes' correctamente.**
-- Ahora sí podemos ver las filas exactas que se eliminaron en la última operación.
-- Reemplace 'X' con la versión ANTERIOR al DELETE y 'Y' con la versión DEL DELETE.
SELECT 
  *, 
  _commit_version as version_modificacion, 
  _commit_timestamp as fecha_modificacion 
FROM table_changes('curso_arquitecturas.pedidos_bronze', 5, 8); -- Ejemplo: table_changes('...', 2, 3)


**Caso de uso:** Auditoría fina. Identificar qué datos específicos fueron alterados por una operación, por ejemplo, para un reporte de cumplimiento de normativas.

### **Ejemplo 4: Restaurar una Tabla a una Versión Anterior (`RESTORE`)**

Si cometiste un error, no necesitas escribir código complejo para arreglarlo. `RESTORE` revierte la tabla a una versión anterior, creando una nueva transacción de restauración.


In [0]:
%sql
-- ¡Vamos a deshacer el borrado que hicimos en el Ejemplo 1!
-- Esto restaurará la tabla al estado de la versión 0.
RESTORE TABLE curso_arquitecturas.pedidos_bronze TO VERSION AS OF 0;

-- Verifiquemos el historial para ver la nueva entrada de 'RESTORE'.
DESCRIBE HISTORY curso_arquitecturas.pedidos_bronze;

**Caso de uso:** Recuperación de desastres de forma rápida y segura. Es la forma oficial de "deshacer" una mala operación.

### **Ejemplo 5: Auditoría de Operaciones a Nivel de Archivo**

El historial también nos dice qué archivos de datos (Parquet) fueron añadidos o eliminados en cada transacción. Esto es útil para una depuración a bajo nivel.

In [0]:
%sql
-- Miremos el historial de la tabla de devoluciones y veamos las métricas de la operación de escritura.
SELECT 
  version, 
  timestamp,
  operation,
  operationMetrics.numFiles AS archivos_escritos,
  operationMetrics.numOutputRows AS filas_escritas,
  operationMetrics.numOutputBytes AS bytes_escritos
FROM (DESCRIBE HISTORY curso_arquitecturas.devoluciones_bronze);

**Caso de uso:** Optimización de rendimiento. Si ves que una operación está generando miles de archivos pequeños, es una señal de que necesitas optimizar la escritura (con técnicas como `OPTIMIZE` o ajustando el tamaño de los archivos).

### **Ejemplo 6: Auditoría Forense - Identificar el Notebook o Job que Modificó los Datos**

**Caso de uso:** En un entorno colaborativo, múltiples procesos y usuarios pueden modificar una tabla. Si ocurre un error, no basta con saber *quién* hizo el cambio, sino *desde dónde* (qué notebook o qué job automatizado). Esta información está anidada en el historial.

**Código:**

In [0]:
%sql
-- Extraemos detalles del contexto de la ejecución de cada transacción.
-- 'notebook.notebookId' es extremadamente útil para la depuración.
SELECT 
  version,
  timestamp,
  userName AS usuario,
  operation,
  operationParameters.mode,
  notebook.notebookId AS id_del_notebook,
  clusterId as id_del_cluster
FROM 
  (DESCRIBE HISTORY curso_arquitecturas.pedidos_bronze);


**Análisis del Resultado:**
Esta consulta enriquece el historial estándar. La columna `id_del_notebook` te da un enlace directo al notebook que ejecutó la transacción, permitiéndote ir directamente al código fuente que causó el cambio. Es una de las herramientas de depuración más potentes en Databricks.

---
### **Interpretación del Historial de la Tabla**

Aquí se desglosa la secuencia de eventos que ocurrieron, narrando la historia de la tabla desde su creación hasta su estado más reciente.

* **Versión 0 (12 de Agosto): Creación Inicial**
    La tabla fue creada por primera vez. Este fue el punto de partida.

* **Versión 1 (14 de Agosto): Reemplazo Total de la Tabla**
    Dos días después, la tabla fue completamente reemplazada usando `CREATE OR REPLACE TABLE`. Esto significa que todo el contenido de la versión 0 fue descartado, y la tabla comenzó de nuevo. Este evento marca el inicio de la sesión de trabajo principal.

* **Versiones 2 y 3: Modificación y Mantenimiento**
    Inmediatamente después del reemplazo, se realizó una operación de borrado (`DELETE`). Justo después, se ejecutó un comando `OPTIMIZE`, que es una operación de mantenimiento para compactar archivos pequeños y mejorar el rendimiento de lectura.

* **Versiones 4 y 5: Habilitación de CDF y Nuevo Borrado**
    La operación `SET TBLPROPERTIES` probablemente fue para habilitar el **Change Data Feed (CDF)** (`delta.enableChangeDataFeed`). Inmediatamente después, se realizó otro `DELETE`, seguramente para probar la funcionalidad de `table_changes` que depende de CDF.

* **Versiones 6, 7 y 8: Ciclo Adicional de Mantenimiento y Pruebas**
    El patrón se repite: una optimización (`OPTIMIZE`), seguida de otra modificación de propiedades y un `DELETE` final. Esto refuerza la idea de que se estaba llevando a cabo una sesión de pruebas intensiva.

* **Versiones 9 y 10: Recuperación de Datos (`RESTORE`)**
    Estos son los eventos más significativos. Las dos operaciones `RESTORE` consecutivas indican que el usuario revirtió la tabla a un estado anterior, deshaciendo efectivamente uno o más de los borrados anteriores. Esto confirma que las operaciones `DELETE` eran parte de un experimento o un error que se necesitaba corregir.

---
### **Observaciones Clave 🔬**

1.  **Ausencia del `id_del_notebook`**: El hecho de que `id_del_notebook` sea `null` en todas las operaciones es una pista muy importante. Generalmente, esto significa que los comandos SQL no se ejecutaron desde un notebook de Databricks (con un clúster de computación), sino directamente a través del **Editor de SQL de Databricks** conectado a un **SQL Warehouse**.

2.  **Sesión de Pruebas Intensiva**: La rápida sucesión de operaciones variadas (`DELETE`, `OPTIMIZE`, `SET TBLPROPERTIES`, `RESTORE`) en un corto período de tiempo (la mayoría en menos de 30 minutos el 14 de agosto) es un claro indicador de una sesión de desarrollo o depuración, no de un pipeline de producción automatizado.

3.  **Uso de Funcionalidades Avanzadas**: El historial demuestra un uso práctico de las características que hacen poderoso a Delta Lake: la capacidad de modificar, optimizar, habilitar funciones y, lo más importante, restaurar la tabla a un punto anterior en el tiempo de forma segura.

El historial narra lo que el ingeniero de datos hizo desde el 14 de agosto de 2025, sometió la tabla a una serie de pruebas rigurosas para validar o experimentar con las capacidades de modificación y recuperación de Delta Lake, probablemente trabajando desde el entorno de Databricks SQL.

### **Ejemplo 7: Análisis Comparativo - Ver el "Antes y Después" de una Fila**

**Caso de uso:** Se ha ejecutado una operación `MERGE` para actualizar precios o estados, y necesitas validar que los cambios se aplicaron correctamente, comparando el valor antiguo con el nuevo para filas específicas.

**Código:**

In [0]:
%sql
-- Paso 1: Simular una actualización en la tabla de productos.
-- Actualizaremos el precio de un producto y la categoría de otro.
UPDATE curso_arquitecturas.productos_bronze 
SET precio_unitario = 99.99 
WHERE id_producto = 2001;

In [0]:
%sql
-- Paso 2: Obtener la última versión del historial para comparar.
-- Asumamos que la operación UPDATE fue la versión 2. La versión anterior era la 1.

DESCRIBE HISTORY curso_arquitecturas.productos_bronze


In [0]:
%sql
-- Paso 3: Comparar los datos de la misma tabla en dos momentos del tiempo.
SELECT
  antes.id_producto,
  antes.nombre_producto,
  antes.precio_unitario AS precio_anterior,
  despues.precio_unitario AS precio_nuevo,
  antes.categoria AS categoria_anterior,
  despues.categoria AS categoria_nueva
FROM 
  curso_arquitecturas.productos_bronze VERSION AS OF 1 AS antes
INNER JOIN 
  curso_arquitecturas.productos_bronze AS despues 
  ON antes.id_producto = despues.id_producto
WHERE 
  antes.precio_unitario <> despues.precio_unitario OR antes.categoria <> despues.categoria;

**Análisis del Resultado:**
Esta consulta te mostrará únicamente las filas que han sufrido una modificación entre las dos versiones, presentando el valor antiguo y el nuevo en la misma línea. Es un método extremadamente eficaz para validar la lógica de negocio en operaciones de `UPDATE` o `MERGE`.


### **Ejemplo 8: Creación de un Log de Auditoría Permanente**

**Caso de uso:** El resultado de `DESCRIBE HISTORY` es una consulta temporal. Para un cumplimiento normativo o una auditoría a largo plazo, necesitas almacenar este historial de forma permanente en su propia tabla Delta.

**Código:**

In [0]:
# Usamos PySpark para mayor flexibilidad al manejar el DataFrame del historial.

# Paso 1: Ejecutar DESCRIBE HISTORY y cargarlo en un DataFrame.
history_df = spark.sql("DESCRIBE HISTORY curso_arquitecturas.pedidos_bronze")

# Paso 2: Seleccionar y aplanar las columnas que nos interesan.
audit_log_df = history_df.select(
    "version",
    "timestamp",
    "userId",
    "userName",
    "operation",
    "operationParameters",
    "notebook.notebookId"
)

# Paso 3: Guardar este log en una nueva tabla Delta.
# Usamos el modo 'append' para poder ejecutar esto periódicamente y añadir el nuevo historial.
audit_log_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("log_auditoria_pedidos")

print("Log de auditoría guardado exitosamente.")

# Consultar el log permanente.
display(spark.sql("SELECT * FROM log_auditoria_pedidos ORDER BY version DESC"))

**Análisis del Resultado:**
Has creado una tabla `log_auditoria_pedidos` que acumula el historial de cambios de tu tabla principal. Puedes construir alertas o informes sobre esta tabla para monitorear actividades sospechosas o simplemente para tener un registro inmutable de todas las transacciones a lo largo del tiempo.